In [2]:
# Load Libraries - Make sure to run this cell!
import pandas as pd
import numpy as np
import re, os
from string import printable
from sklearn import model_selection

#import gensim
import tensorflow as tf
from keras.models import Sequential, Model, model_from_json, load_model
from keras import regularizers
from keras.layers import Dense, Dropout, Activation, Lambda, Flatten, Concatenate
from keras.layers import Input, ELU, LSTM, Embedding, Convolution2D, MaxPooling2D, \
BatchNormalization, Convolution1D, MaxPooling1D
from keras.preprocessing import sequence
from keras.optimizers import SGD, Adam, RMSprop
import np_utils
from keras import initializers
from keras import backend as K

from pathlib import Path
import json

import warnings
warnings.filterwarnings("ignore")


<img src="../img/GTK_Logo_Social Icon.jpg" width=175 align="right" />

## Featureless Deep Learning

This notebook shows three commonly used neural network architectures to detect malicious URLs using **featureless Deep Learning**. [Keras](https://keras.io/) is used as high-level API for [tensorflow](https://www.tensorflow.org/) backend). 

Please refer to the slides (```Module 4.4 Featureless Deep Learning```) for additional info!


## Preprocess raw URLs

In [3]:
## Load data URL

DATA_HOME = '../data/'
df = pd.read_csv(DATA_HOME + 'url_data_mega_deep_learning.csv')
df.sample(n=25).head(25) 

,url,isMalicious
122529,hcorganize.com/kontrol/assets/plugins/fullcale...,1
108721,igforweddingpros.com/Docs/Folders/dropbox,1
179608,hotnepalijobs.com/2015/06/banking-job-vacancy-...,1
45959,rotoworld.com/player/mlb/4972/player?r=1,0
84937,scarymommy.com/circle-with-disney-screen-time,0
92374,agrupamentotabuaco.com/images/20142015/PDF/201...,0
187812,hc0.me/panel/?login,1
90670,thethirdmedia.com/Article/201703/show376242c77...,0
142285,zorattoproductions.com/up,1
17975,gree.com.cn/pczwb/xwzx/cms_category_1261/20170...,0


In [4]:
# Initial Data Preparation URL

# Step 1: Convert raw URL string in list of lists where characters that are contained in "printable" are stored encoded as integer 
url_int_tokens = [[printable.index(x) + 1 for x in url if x in printable] for url in df.url]

# Step 2: Cut URL string at max_len or pad with zeros if shorter
max_len=75
X = sequence.pad_sequences(url_int_tokens, maxlen=max_len)
 
# Step 3: Extract labels form df to numpy array
target = np.array(df.isMalicious)

print('Matrix dimensions of X: ', X.shape, 'Vector dimension of target: ', target.shape)

Matrix dimensions of X:  (194798, 75) Vector dimension of target:  (194798,)


In [5]:
# Simple Cross-Validation: Split the data set into training and test data
X_train, X_test, target_train, target_test = model_selection.train_test_split(X, target, test_size=0.25, random_state=33)

In [6]:
# GENERAL get layer dimensions for any model!
def print_layers_dims(model):
    l_layers = model.layers
    # Note None is ALWAYS batch_size
    for i in range(len(l_layers)):
        print(l_layers[i])
        print('Input Shape: ', l_layers[i].input_shape, 'Output Shape: ', l_layers[i].output_shape)

# GENERAL save model to disk function!
def save_model(fileModelJSON,fileWeights):
    #print("Saving model to disk: ",fileModelJSON,"and",fileWeights)
    #have h5py installed
    if Path(fileModelJSON).is_file():
        os.remove(fileModelJSON)
    json_string = model.to_json()
    with open(fileModelJSON,'w' ) as f:
        json.dump(json_string, f)
    if Path(fileWeights).is_file():
        os.remove(fileWeights)
    model.save_weights(fileWeights)
    

# GENERAL load model from disk function!
def load_model(fileModelJSON,fileWeights):
    #print("Saving model to disk: ",fileModelJSON,"and",fileWeights)
    with open(fileModelJSON, 'r') as f:
         model_json = json.load(f)
         model = model_from_json(model_json)
    
    model.load_weights(fileWeights)
    return model

## Architecture 1 - Simple LSTM

In [7]:
## Deep Learning model Definition --- A --- (Simple LSTM)


def simple_lstm(max_len=75, emb_dim=32, max_vocab_len=100, lstm_output_size=32, W_reg=regularizers.l2(1e-4)):
    # Input
    main_input = Input(shape=(max_len,), dtype='int32', name='main_input')
    # Embedding layer
    emb = Embedding(input_dim=max_vocab_len, output_dim=emb_dim, input_length=max_len)(main_input) 
    
    # LSTM layer
    lstm = LSTM(lstm_output_size)(emb)
    lstm = Dropout(0.5)(lstm)
    
    # Output layer (last fully connected layer)
    output = Dense(1, activation='sigmoid', name='output')(lstm)

    # Compile model and define optimizer
    model = Model(inputs=[main_input], outputs=[output])
    adam = tf.keras.optimizers.legacy.Adam(learning_rate=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [8]:
# Fit model and Cross-Validation, ARCHITECTURE 1 SIMPLE LSTM
nb_epoch = 3
batch_size = 32

model = simple_lstm()
model.fit(X_train, target_train, epochs=nb_epoch, batch_size=batch_size)
loss, accuracy = model.evaluate(X_test, target_test, verbose=1)

print('\nFinal Cross-Validation Accuracy', accuracy, '\n')
print_layers_dims(model)

Epoch 1/3
4566/4566 [==============================] - 45s 10ms/step - loss: 0.4692 - accuracy: 0.7848
Epoch 2/3
4566/4566 [==============================] - 44s 10ms/step - loss: 0.3963 - accuracy: 0.8289
Epoch 3/3
1522/1522 [==============================] - 4s 2ms/step - loss: 0.3654 - accuracy: 0.8405

Final Cross-Validation Accuracy 0.8405338525772095 

Input Shape:  [(None, 75)] Output Shape:  [(None, 75)]
Input Shape:  (None, 75) Output Shape:  (None, 75, 32)
Input Shape:  (None, 75, 32) Output Shape:  (None, 32)
Input Shape:  (None, 32) Output Shape:  (None, 32)
Input Shape:  (None, 32) Output Shape:  (None, 1)


In [11]:
model_name = "deeplearning_LSTM"
save_model(DATA_HOME + model_name + ".json", DATA_HOME + model_name + ".h5")
model = load_model(DATA_HOME + model_name + ".json", DATA_HOME + model_name + ".h5")
#print_layers_dims(model)

## Architecture 2 - 1D Convolution and LSTM

In [31]:
## Deep Learning model Definition --- B --- (1D Convolution and LSTM)

def lstm_conv(max_len=75, emb_dim=32, max_vocab_len=100, lstm_output_size=32, W_reg=regularizers.l2(1e-4)):
    # Input
    main_input = Input(shape=(max_len,), dtype='int32', name='main_input')
    # Embedding layer
    emb = Embedding(input_dim=max_vocab_len, output_dim=emb_dim, input_length=max_len,
                activity_regularizer=W_reg)(main_input) 
    emb = Dropout(0.25)(emb)

    # Conv layer
    conv = Convolution1D(filters=256, \
                     kernel_size=5)(emb)
    conv = ELU()(conv)

    conv = MaxPooling1D(pool_size=4)(conv)
    #conv = BatchNormalization(mode=0)(conv)
    conv = Dropout(0.5)(conv)

    # LSTM layer
    lstm = LSTM(lstm_output_size)(conv)
    lstm = Dropout(0.5)(lstm)
    
    # Output layer (last fully connected layer)
    output = Dense(1, activation='sigmoid', name='output')(lstm)

    # Compile model and define optimizer
    model = Model([main_input], [output])
    adam = tf.keras.optimizers.legacy.Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [32]:
# Fit model and Cross-Validation, ARCHITECTURE 2 CONV + LSTM
nb_epoch = 5
batch_size = 32

model = lstm_conv()
model.fit(X_train, target_train, epochs=nb_epoch, batch_size=batch_size)
loss, accuracy = model.evaluate(X_test, target_test, verbose=1)

print('\nFinal Cross-Validation Accuracy', accuracy, '\n')
print_layers_dims(model)

Epoch 1/5
4566/4566 [==============================] - 51s 11ms/step - loss: 0.4834 - accuracy: 0.7653
Epoch 2/5
4566/4566 [==============================] - 55s 12ms/step - loss: 0.3691 - accuracy: 0.8418
Epoch 3/5
4566/4566 [==============================] - 60s 13ms/step - loss: 0.3318 - accuracy: 0.8600
Epoch 4/5
4566/4566 [==============================] - 60s 13ms/step - loss: 0.3104 - accuracy: 0.8698
Epoch 5/5
1522/1522 [==============================] - 5s 3ms/step - loss: 0.2519 - accuracy: 0.8973

Final Cross-Validation Accuracy 0.897289514541626 

Input Shape:  [(None, 75)] Output Shape:  [(None, 75)]
Input Shape:  (None, 75) Output Shape:  (None, 75, 32)
Input Shape:  (None, 75, 32) Output Shape:  (None, 75, 32)
Input Shape:  (None, 75, 32) Output Shape:  (None, 71, 256)
Input Shape:  (None, 71, 256) Output Shape:  (None, 71, 256)
Input Shape:  (None, 71, 256) Output Shape:  (None, 17, 256)
Input Shape:  (None, 17, 256) Output Shape:  (None, 17, 256)
Input Shape:  (None, 1

In [33]:
model_name = "deeplearning_1DConvLSTM"
save_model(DATA_HOME + model_name + ".json", DATA_HOME + model_name + ".h5")
model = load_model(DATA_HOME + model_name + ".json", DATA_HOME + model_name + ".h5")
#print_layers_dims(model)

## Architecture 3 - 1D Convolutions and Fully Connected Layers

In [52]:
## Deep Learning model Definition --- C --- (1D Convolutions and Fully Connected Layers)

def conv_fully(max_len=75, emb_dim=32, max_vocab_len=100, W_reg=regularizers.l2(1e-4)):
    # Input
    main_input = Input(shape=(max_len,), dtype='int32', name='main_input')
    # Embedding layer
    emb = Embedding(input_dim=max_vocab_len, output_dim=emb_dim, input_length=max_len,
                activity_regularizer=W_reg)(main_input) 
    emb = Dropout(0.25)(emb)

    
    def sum_1d(X):
        return K.sum(X, axis=1)
    
    def get_conv_layer(emb, filter_length=5, nb_filter=256):
        # Conv layer
        conv = Convolution1D(kernel_size=filter_length, filters=nb_filter)(emb)
        conv = ELU()(conv)

        conv = Lambda(sum_1d, output_shape=(nb_filter,))(conv)
        #conv = BatchNormalization(mode=0)(conv)
        conv = Dropout(0.5)(conv)
        return conv
        
    # Multiple Conv Layers
    
    # calling custom conv function from above
    conv1 = get_conv_layer(emb, filter_length=2, nb_filter=256)
    conv2 = get_conv_layer(emb, filter_length=3, nb_filter=256)
    conv3 = get_conv_layer(emb, filter_length=4, nb_filter=256)
    conv4 = get_conv_layer(emb, filter_length=5, nb_filter=256)

    # Fully Connected Layers
    merged = tf.keras.layers.Concatenate()([conv1,conv2,conv3,conv4])

    hidden1 = Dense(1024)(merged)
    hidden1 = ELU()(hidden1)
    hidden1 = BatchNormalization()(hidden1)
    hidden1 = Dropout(0.5)(hidden1)

    hidden2 = Dense(1024)(hidden1)
    hidden2 = ELU()(hidden2)
    hidden2 = BatchNormalization()(hidden2)
    hidden2 = Dropout(0.5)(hidden2)
    
    # Output layer (last fully connected layer)
    output = Dense(1, activation='sigmoid', name='output')(hidden2)

    # Compile model and define optimizer
    model = Model([main_input], [output])
    adam = tf.keras.optimizers.legacy.Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [54]:
# Fit model and Cross-Validation, ARCHITECTURE 3 CONV + FULLY CONNECTED
nb_epoch = 5
batch_size = 32

model = conv_fully()
model.fit(X_train, target_train, epochs=nb_epoch, batch_size=batch_size)
loss, accuracy = model.evaluate(X_test, target_test, verbose=1)

print('\nFinal Cross-Validation Accuracy', accuracy, '\n')
print_layers_dims(model)

Epoch 1/5
4566/4566 [==============================] - 83s 18ms/step - loss: 0.6324 - accuracy: 0.6902
Epoch 2/5
4566/4566 [==============================] - 83s 18ms/step - loss: 0.4834 - accuracy: 0.7685
Epoch 3/5
4566/4566 [==============================] - 84s 18ms/step - loss: 0.4437 - accuracy: 0.7927
Epoch 4/5
4566/4566 [==============================] - 329s 72ms/step - loss: 0.4273 - accuracy: 0.8020
Epoch 5/5
1522/1522 [==============================] - 10s 7ms/step - loss: 0.3756 - accuracy: 0.8305

Final Cross-Validation Accuracy 0.8304722905158997 

Input Shape:  [(None, 75)] Output Shape:  [(None, 75)]
Input Shape:  (None, 75) Output Shape:  (None, 75, 32)
Input Shape:  (None, 75, 32) Output Shape:  (None, 75, 32)
Input Shape:  (None, 75, 32) Output Shape:  (None, 74, 256)
Input Shape:  (None, 75, 32) Output Shape:  (None, 73, 256)
Input Shape:  (None, 75, 32) Output Shape:  (None, 72, 256)
Input Shape:  (None, 75, 32) Output Shape:  (None, 71, 256)
Input Shape:  (None, 7

In [55]:
# get probabilities of target predictions
target_proba = model.predict(X_test, batch_size=1)

48700/48700 [==============================] - 71s 1ms/step


In [56]:
target_proba[0:10]

array([[0.26679173],
       [0.9480639 ],
       [0.7216458 ],
       [0.65599823],
       [0.87424594],
       [0.02143307],
       [0.34641847],
       [0.99341804],
       [0.02237426],
       [0.28046033]], dtype=float32)

In [61]:
model_name = "deeplearning_1DConv"
save_model(DATA_HOME + model_name + ".json", DATA_HOME + model_name + ".h5")
# model = load_model(DATA_HOME + model_name + ".json", DATA_HOME + model_name + ".h5")
#print_layers_dims(model)

In [62]:
# get for example word2vec embedding weight matix
l_layers = model.layers
weights = l_layers[1].get_weights()
weights[0].shape

(100, 32)

## Making a new prediction

In [83]:
test_url_mal = "naureen.net/etisalat.ae/index2.php"
test_url_benign = "sixt.com/php/reservation?language=en_US"

url = test_url_benign

In [84]:
# Step 1: Convert raw URL string in list of lists where characters that are contained in "printable" are stored encoded as integer 
url_int_tokens = [[printable.index(x) + 1 for x in url if x in printable]]

# Step 2: Cut URL string at max_len or pad with zeros if shorter
max_len=75
X = sequence.pad_sequences(url_int_tokens, maxlen=max_len)

In [85]:
target_proba = model.predict(X, batch_size=5)
def print_result(proba):
    if proba > 0.5:
        return "malicious"
    else:
        return "benign"
print("Test URL:", url, "is", print_result(target_proba[0]))

1/1 [==============================] - 0s 18ms/step
Test URL: sixt.com/php/reservation?language=en_US is benign
